In [12]:
import os
import math

In [7]:
filepath = "./a.txt"

In [8]:
x_o, y_o, z_o = -305.923, -2.358, -798.047
x_x, y_x, z_x = -316.039, -178.714, -530.861
x_y, y_y, z_y = 295.871, 0.771, -794.537

In [9]:
with open(filepath, 'r') as f:
    lines = f.readlines()

In [10]:
points = [line.split()[:3] for line in lines]
for point in points:
    point[0] = float(point[0]) - x_o
    point[1] = float(point[1]) - y_o
    point[2] = float(point[2]) - z_o

In [11]:
with open("./a_differential.txt", 'w') as f:
    for i in range(len(points)):
        f.write(str(points[i][0]) + " " 
                + str(points[i][1]) + " "
                + str(points[i][2]) + "\n"  )

In [13]:
y_r = y_o - y_x
z_r = z_o - z_x
r = abs(math.sqrt((y_r)**2 + (z_r)**2))
cos = y_r / r
sin = z_r / r

In [14]:
#x軸に対して負の回転
res = []
for point in points:
    x, y, z = point
    X = x
    Y = (y*cos) + (z*sin)
    Z = (-y*sin) + (z*cos)
    res.append([X, Y, Z])

In [19]:
x_r = x_o - x_x
y_r = y_o - y_x
r = abs(math.sqrt((x_r)**2 + (y_r)**2))
cos = x_r / r
sin = y_r / r

In [24]:
#z軸に対して負の回転
res_z = []
for point in res:
    x, y, z = point
    Z = z
    X = (x*cos) + (-y*sin)
    Y = (x*sin) + (y*cos)
    res_z.append([X, Y, Z])

In [25]:
with open("./a_rotation_x_z.txt", 'w') as f:
    for i in range(len(res)):
        f.write(str(res_z[i][0]) + " "
                + str(res_z[i][1]) + " "
                + str(res_z[i][2]) + "\n"  )